In [1]:
# method for importing all the necessary modules
def import_modules():
    import matplotlib.pyplot as plt
    import seaborn as sns
    from keras.models import Sequential
    from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
    from keras.preprocessing.image import ImageDataGenerator
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import classification_report,confusion_matrix
    import pandas as pd

    return plt, sns, Sequential, Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization, ImageDataGenerator, train_test_split, classification_report, confusion_matrix, pd



In [2]:
# method for importing the dataset
def import_dataset(filename):
    import pandas as pd
    dataset = pd.read_csv(filename)
    return dataset

In [3]:
# Data Preprocessing

plt, sns, Sequential, Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization, ImageDataGenerator, train_test_split, classification_report, confusion_matrix, pd = import_modules()
train_df = import_dataset('sign_mnist_train.csv')
test_df = import_dataset('sign_mnist_test.csv')
    
y_train = train_df['label']
y_test = test_df['label']
del train_df['label']
del test_df['label']
    
# Label Binarizer to reduce the processing speed. It is used to convert the labels into a matrix
from sklearn.preprocessing import LabelBinarizer
label_binarizer = LabelBinarizer()
y_train = label_binarizer.fit_transform(y_train)
y_test = label_binarizer.fit_transform(y_test)

x_train = train_df.values
x_test = test_df.values

# Normalize the data
x_train = x_train / 255
x_test = x_test / 255

# Reshape the data
x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)


In [4]:
# model creation
model = Sequential()
model.add(Conv2D(75 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = (28,28,1)))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(50 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(25 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Flatten())
model.add(Dense(units = 512 , activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(units = 24 , activation = 'softmax'))

In [5]:
datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False, 
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=10,
        zoom_range = 0.1, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=False,
        vertical_flip=False)

datagen.fit(x_train)

In [7]:
plt, sns, Sequential, Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization, ImageDataGenerator, train_test_split, classification_report, confusion_matrix, pd = import_modules()
model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
model.summary()

history = model.fit(datagen.flow(x_train,y_train, batch_size = 128) ,epochs = 20 , validation_data = (x_test, y_test))
# the model is saved in the file CSE572_DataMining.h5 and can be used for testing
model.save('CSE572_DataMining.h5')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 75)        750       
                                                                 
 batch_normalization (Batch  (None, 28, 28, 75)        300       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 75)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 50)        33800     
                                                                 
 dropout (Dropout)           (None, 14, 14, 50)        0         
                                                                 
 batch_normalization_1 (Bat  (None, 14, 14, 50)        2